# Making Jet clusters and exporting them
## Need Aggregator files, Meta_data_subjects.json made during the aggregation in BoxTheJets
This notebook takes the jets detected per subjects and looks for clusters in space and time. If two jets of different clusters fall within the epsilon given by the user (set by eps and time_eps) they are clustered together to make a jet cluster, this can be repeated such that more jets are added to the cluster. Clusters can only contain one jet per subject such that closeby jets are detected seperatly. 
The second part of this notebook requires the database of the Zooniverse to make the conversion between pixels ans solar coordinates. The meta data is saved in the Meta_data_subjects.json file complied from the solar-jet-hunter-subjects.csv file dowloaded during the aggregation


In [14]:
import os
from aggregation import Aggregator, get_subject_image
from aggregation import SOL
from aggregation import MetaFile
from aggregation import QuestionResult
from aggregation import json_export_list
from aggregation import get_box_edges, sigma_shape
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.dates import DateFormatter
import numpy as np
import tqdm
from scipy.cluster.hierarchy import dendrogram
plt.style.use('default')
%matplotlib inline

In [15]:
aggregator = Aggregator('reductions/point_reducer_hdbscan_box_the_jets.csv',
                        'reductions/shape_reducer_dbscan_box_the_jets.csv')
aggregator.load_extractor_data('extracts/point_extractor_by_frame_box_the_jets.csv',
                               'extracts/shape_extractor_rotateRectangle_box_the_jets.csv')

reducer_data = QuestionResult('../question_reducer_combined_workflows.csv')

sol = SOL('../Meta_data_subjects.json', aggregator)

metafile = MetaFile('../Meta_data_subjects.json')

In [16]:
Jet_clusters = np.array([])
# Set the space and time epsilon
eps, time_eps = 3.0, 2.0

for s in tqdm.tqdm(range(len(metafile.SOL_unique))):
    del_index = np.array([], dtype=int)
    SOL_event = metafile.SOL_unique[s]
    try:
        clusters, distance_met, point_met, box_met = sol.filter_jet_clusters(
            SOL_event, eps=eps, time_eps=time_eps)
    except:
        continue
    for j, cluster in enumerate(clusters):
        cluster.adding_new_attr("SOL", SOL_event)
        if len(cluster.jets) == 1 and reducer_data.Agr_mask(reducer_data.get_data_by_id(cluster.jets[0].subject))[-1][0] == 'n':
            # jets that only last 1 subject and do not have 50% agreement yes are excluded
            del_index = np.append(del_index, j)
    if len(del_index) > 0:
        # print(f'Remove {len(del_index)} clusters from list due to too low agreement')
        clusters = np.delete(clusters, del_index)
    Jet_clusters = np.append(Jet_clusters, clusters)

  0%|          | 0/365 [00:00<?, ?it/s]

  1%|          | 2/365 [00:00<01:44,  3.48it/s]/Users/pjol/Documents/SolarJets-1/BoxTheJets/aggregation/SOL_class.py:419: RuntimeWarning: divide by zero encountered in scalar divide
  point_metric[k, j] = point_dist / \
100%|██████████| 365/365 [03:02<00:00,  2.00it/s]


### In this next part we will extract the Solar coordinates for the subject values. We use the Meta_data_subjects.json file to obtain the needed information.

In [17]:
from aggregation.image_handler import solar_conversion


def get_solar_distance(subject_id, pair, metadata):
    '''
        Get the solar projected distance between the two pairs of X,Y coordinates
            Inputs:
            -------
            subject_id : int
                subject_id used in the Zooniverse subject
            pair : np.array
                x,y Coordinates of the two points 1,2 for which the solar distance needs to be calculated
                format [[x1,y1],[x2,y2]]
    '''
    solw1 = solar_conversion(subject_id, pair[0][0], pair[0][1], metadata)
    solw2 = solar_conversion(subject_id, pair[1][0], pair[1][1], metadata)
    # Euclidean distance
    distance = np.sqrt((solw1[0]-solw2[0])**2 + (solw1[1]-solw2[1])**2)
    return distance

### Go through the list of jet clusters and determine their propeties in physical coordinates

In [18]:
import logging

ID = 1

for C in tqdm.tqdm(Jet_clusters):
    # print('Jet start')
    H = np.array([])
    W = np.array([])
    X = np.array([])
    Y = np.array([])
    sig = np.array([])
    H_sig = np.zeros((len(C.jets), 2))
    obs_time = np.array([], dtype='datetime64')
    end_time = np.array([], dtype='datetime64')
    for j, jet in enumerate(C.jets):
        # print(j, len(C.jets))
        width_pair, height_pair = jet.get_width_height_pairs()
        # Find sigma of maximum height by first getting the pixel height
        H_pix_box = np.sqrt((height_pair[1][0]-height_pair[0][0]) **
                            2 + (height_pair[1][1]-height_pair[0][1])**2)
        index = list(map(int, jet.cluster_values)).index(int(H_pix_box))
        # Get the height of the box in pixels for the +-1 sigma
        plus_sigma, minus_sigma = sigma_shape(jet.cluster_values, jet.sigma)
        H_pix_minus = minus_sigma[index]
        H_pix_plus = plus_sigma[index]
        # print(width_pair,height_pair)
        # Get the solar locations on the jet
        metadata = metafile.get_subjectdata_by_id(jet.subject)
        # file=metadata['#file_name_0']
        try:
            Bx, By = solar_conversion(jet.subject, jet.start[0], jet.start[1], metadata)
        except:
            print('This one breaks', jet.subject)
            continue
        logging.getLogger('sunpy').setLevel(logging.CRITICAL)
        Ex, Ey = solar_conversion(jet.subject, jet.end[0], jet.end[1], metadata)
        # print('Start base',Bx,By)
        # print('sigma',jet.sigma)
        # Add as attributes and as a list
        jet.adding_new_attr("solar_start", [Bx, By])
        jet.adding_new_attr("solar_end", [Ex, Ey])
        sig = np.append(sig, jet.sigma)
        X = np.append(X, Bx)
        Y = np.append(Y, By)
        # Get the dates the subjecst were observed
        O = metafile.get_subjectkeyvalue_by_id(jet.subject, 'startDate')
        obs_time = np.append(obs_time, O)
        E = metafile.get_subjectkeyvalue_by_id(jet.subject, 'endDate')
        end_time = np.append(end_time, E)
        # Calculate the height an wisth in arcsec
        height = get_solar_distance(jet.subject, height_pair, metadata)
        width = get_solar_distance(jet.subject, width_pair, metadata)
        # Add as attributes and list
        jet.adding_new_attr("solar_H", height)
        jet.adding_new_attr("solar_W", width)
        H = np.append(H, height)
        W = np.append(W, width)
        # Get the error on the height by scaling the height with the (height_sigma/height -1)
        err_plus, err_minus = height*(H_pix_plus/H_pix_box-1), height*(H_pix_minus/H_pix_box-1)
        H_sig[j] = np.array([err_plus, err_minus])
        jet.adding_new_attr("solar_H_sig", [err_plus, err_minus])

    duration = (end_time[-1]-obs_time[0])/np.timedelta64(1, 'm')
    if obs_time[np.argmax(H)] == obs_time[0]:
        vel = np.NaN
    else:
        vel = np.max(H)/((obs_time[np.argmax(H)]-obs_time[0]) / np.timedelta64(1, 's'))

    C.adding_new_attr("ID", ID)
    C.adding_new_attr('Max_Height', np.max(H))
    C.adding_new_attr('std_maxH', H_sig[np.argmax(H)])
    C.adding_new_attr("Height", np.average(H))
    C.adding_new_attr("std_H", np.std(H))
    C.adding_new_attr("Width", np.average(W))
    C.adding_new_attr("std_W", np.std(W))
    C.adding_new_attr("Bx", np.average(X))
    C.adding_new_attr("std_Bx", np.std(X))
    C.adding_new_attr("By", np.average(Y))
    C.adding_new_attr("std_By", np.std(Y))
    C.adding_new_attr("obs_time", obs_time[0])
    C.adding_new_attr("sigma", np.average(sig))
    C.adding_new_attr("Duration", duration)
    C.adding_new_attr("Velocity", vel)

    ID += 1

  0%|          | 0/883 [00:00<?, ?it/s]

100%|██████████| 883/883 [02:37<00:00,  5.61it/s]


### Add the longitude and latitude of the measured basepoints as properties to the Jet_cluster objects

In [19]:
import astropy.units as u
from astropy.coordinates import SkyCoord

import sunpy.map
from sunpy.coordinates import frames

In [20]:
for C in tqdm.tqdm(Jet_clusters):
    # print(C.Bx,C.By)
    X, Y = C.Bx, C.By
    sky_coord = SkyCoord(X*u.arcsec, Y*u.arcsec, frame=frames.Helioprojective(observer="earth",
                                                                              obstime=str(C.obs_time)))
    # print(sky_coord.heliographic_stonyhurst)
    Coord = sky_coord.heliographic_stonyhurst
    if np.isnan(Coord.lat):
        # print('Coordinates off limb')
        with frames.Helioprojective.assume_spherical_screen(sky_coord.observer):
            # print(sky_coord.heliographic_stonyhurst)
            Coord = sky_coord.heliographic_stonyhurst
            C.adding_new_attr("Lat", float(str(Coord.lat).split('d')[0]))
            C.adding_new_attr("Lon", float(str(Coord.lon).split('d')[0]))

    else:
        C.adding_new_attr("Lat", float(str(Coord.lat).split('d')[0]))
        C.adding_new_attr("Lon", float(str(Coord.lon).split('d')[0]))

100%|██████████| 883/883 [00:04<00:00, 193.98it/s]


## Flagging
We introduce three binary flags to indicate a higher uncertainty of our jet clusters. 
flag 100 means the jet cluster has a duration of less than 6 minutes, which for many corresponds to a jet cluster found in one Zooniverse subject. 
flag 010 means the velocity estimate could not be calculated because the maximum was reached in the first subject the jet was found in. 
flag 001 means the basepoint has a Longitude of higher than 90 degrees meaning the base point was found to be (slightly) off limb. 


In [21]:
# Flagging
stat_dur = np.array([Jet_clusters[i].Duration for i in range(len(Jet_clusters))], dtype=float)
stat_vel = np.array([Jet_clusters[i].Velocity for i in range(len(Jet_clusters))], dtype=float)
stat_Lon = np.array([Jet_clusters[i].Lon for i in range(len(Jet_clusters))], dtype=float)

flag1 = np.where(stat_dur < 6)[0]
flag2 = np.where(np.isnan(stat_vel))[0]
flag3 = np.where(np.abs(stat_Lon) > 90)[0]
tel = 0
flags = np.array([])

for i in range(len(Jet_clusters)):
    f1, f2, f3 = i in flag1, i in flag2, i in flag3
    if f1 or f2 or f3:
        flag = str(int(f1 == True))+str(int(f2 == True))+str(int(f3 == True))
    else:
        flag = '000'
        tel += 1
    flags = np.append(flags, flag)

print('Amount Jet clusters with flags', tel)

Amount Jet clusters with flags 339


In [22]:
# Add flags as a attribute
for i, C in enumerate(Jet_clusters):
    C.flag = flags[i]

## Export the results of the clustering
Export the JetCluster objects to a JSON file
or 
Export the results to a csv file 

In [23]:
def make_export_folder():
    path = 'exports/'
    # check if folder for plots exists
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
        print("exports directory is created")


make_export_folder()

In [24]:
# Export all the JetCluster objects
json_export_list(Jet_clusters, f'exports/Jet_clusters_{eps}_{time_eps}_paper')
# Jet_clusters[0].json_export('output_single') #Export a single JetCluster object

The 883 JetCluster objects are exported to exports/Jet_clusters_3.0_2.0_paper.json.


In [25]:
Cluster_date = np.array([Jet_clusters[i].obs_time for i in range(len(Jet_clusters))], dtype=str)
Cluster_SOL = np.array([Jet_clusters[i].SOL for i in range(len(Jet_clusters))], dtype=str)
stat_Bx = np.array([Jet_clusters[i].Bx for i in range(len(Jet_clusters))], dtype=str)
stat_By = np.array([Jet_clusters[i].By for i in range(len(Jet_clusters))], dtype=str)
stat_Lon = np.array([Jet_clusters[i].Lon for i in range(len(Jet_clusters))], dtype=str)
stat_Lat = np.array([Jet_clusters[i].Lat for i in range(len(Jet_clusters))], dtype=str)
stat_H = np.array([Jet_clusters[i].Max_Height for i in range(len(Jet_clusters))], dtype=str)
stat_W = np.array([Jet_clusters[i].Width for i in range(len(Jet_clusters))], dtype=str)
stat_dur = np.array([Jet_clusters[i].Duration for i in range(len(Jet_clusters))], dtype=str)
stat_vel = np.array([Jet_clusters[i].Velocity for i in range(len(Jet_clusters))], dtype=str)
stat_sigma = np.array([Jet_clusters[i].sigma for i in range(len(Jet_clusters))], dtype=str)
std_H = np.array([Jet_clusters[i].std_maxH for i in range(len(Jet_clusters))], dtype=str)
std_W = np.array([Jet_clusters[i].std_W for i in range(len(Jet_clusters))], dtype=str)
std_Bx = np.array([Jet_clusters[i].std_Bx for i in range(len(Jet_clusters))], dtype=str)
std_By = np.array([Jet_clusters[i].std_By for i in range(len(Jet_clusters))], dtype=str)

In [26]:
csvfile = open(f'exports/Jet_clusters_{eps}_{time_eps}_paper.csv', 'w')
csvfile.writelines('#date, SOL_event, duration, basepoint_X, std_X, basepoint_Y, std_Y, basepoint_X_longitude, basepoint_Y_latitude, max_height, upper_H, lower_H, avg_width, std_width, velocity, sigma, flags')
csvfile.writelines('\n')
with open(f'exports/Jet_clusters_{eps}_{time_eps}_paper.csv', 'a') as csvfile:
    np.savetxt(csvfile, np.column_stack((Cluster_date, Cluster_SOL, stat_dur, stat_Bx, std_Bx, stat_By, std_By, stat_Lon,
               stat_Lat, stat_H, std_H, stat_W, std_W, stat_vel, stat_sigma, flags)), delimiter=",", newline='\n', fmt='%s')
csvfile.close()